<a href="https://colab.research.google.com/github/edzimmermann/NLP/blob/main/cbow_glove.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import pytorch libraries
%matplotlib inline
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

from torch.utils.data import Dataset, DataLoader

# Text Classification
In this part of the tutorial we develop a continuous bag of words (CBOW) model for a text classification task described [here]( https://people.cs.umass.edu/~miyyer/pubs/2015_acl_dan.pdf). The CBOW model was first described [here](https://arxiv.org/pdf/1301.3781.pdf)

## Subjectivity Dataset
The subjectivity dataset has 5000 subjective and 5000 objective processed sentences. To get the data:
```
wget http://www.cs.cornell.edu/people/pabo/movie-review-data/rotten_imdb.tar.gz
```

In [ ]:
def unpack_dataset():
    ! wget http://www.cs.cornell.edu/people/pabo/movie-review-data/rotten_imdb.tar.gz
    ! mkdir data
    ! tar -xvf rotten_imdb.tar.gz -C data

In [ ]:
#unpack_dataset()

In [ ]:
!ls data

ls: cannot access 'data': No such file or directory


In [ ]:
! head -2 data/plot.tok.gt9.5000

head: cannot open 'data/plot.tok.gt9.5000' for reading: No such file or directory


In [ ]:
from pathlib import Path
PATH = Path("data")
list(PATH.iterdir())

FileNotFoundError: [Errno 2] No such file or directory: 'data'

## Tokenization
Tokenization is the task of chopping up text into pieces, called tokens.

spaCy is an open-source software library for advanced Natural Language Processing. Here we will use it for tokenization.  

### Simple Tokenization

In [ ]:
# We need each line in the file
def read_file(path):
    """ Read file returns a list of lines.
    """
    with open(path, encoding = "ISO-8859-1") as f:
        content = f.readlines()
    return content

In [ ]:
obj_lines = read_file(PATH/"plot.tok.gt9.5000")

In [ ]:
obj_lines[0]

In [ ]:
np.array(obj_lines[0].strip().lower().split(" "))

### Much better tokenization with Spacy

In [ ]:
#!pip install -U spacy

In [ ]:
import spacy

In [ ]:
# first time run this
#!python3 -m spacy download en

In [ ]:
tok = spacy.load('en')

In [ ]:
obj_lines = read_file(PATH/"plot.tok.gt9.5000")

In [ ]:
len(obj_lines)

In [ ]:
obj_lines[0]

In [ ]:
test = tok(obj_lines[0])

In [ ]:
np.array([x for x in test])

## Split dataset in train and validation

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
sub_content = read_file(PATH/"quote.tok.gt9.5000")
obj_content = read_file(PATH/"plot.tok.gt9.5000")
sub_content = np.array([line.strip().lower() for line in sub_content])
obj_content = np.array([line.strip().lower() for line in obj_content])
sub_y = np.zeros(len(sub_content))
obj_y = np.ones(len(obj_content))
X = np.append(sub_content, obj_content)
y = np.append(sub_y, obj_y)

In [ ]:
X[0], y[0]

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train[:5], y_train[:5]

## Word to index mapping
In interest of time we will tokenize without spaCy. Here we will compute a vocabulary of words based on the training set and a mapping from word to an index.

In [ ]:
from collections import defaultdict

In [ ]:
def get_vocab(content):
    """Computes Dict of counts of words.

    Computes the number of times a word is on a document.
    """
    vocab = defaultdict(float)
    for line in content:
        words = set(line.split())
        for word in words:
            vocab[word] += 1
    return vocab

In [ ]:
#Getting the vocabulary from the training set
word_count = get_vocab(X_train)

In [ ]:
#word_count

In [ ]:
len(word_count.keys())

In [ ]:
# let's delete words that are very infrequent
for word in list(word_count):
    if word_count[word] < 5:
        del word_count[word]

In [ ]:
len(word_count.keys())

In [ ]:
## Finally we need an index for each word in the vocab
vocab2index = {"<PAD>":0, "UNK":1} # init with padding and unknown
words = ["<PAD>", "UNK"]
for word in word_count:
    vocab2index[word] = len(words)
    words.append(word)

In [ ]:
#vocab2index

## Sentence encoding
Here we encode each sentence as a sequence of indices corresponding to each word.

In [ ]:
x_train_len = np.array([len(x.split()) for x in X_train])
x_val_len = np.array([len(x.split()) for x in X_val])

In [ ]:
np.percentile(x_train_len, 95) # let set the max sequence len to N=40

In [ ]:
X_train[0]

In [ ]:
# returns the index of the word or the index of "UNK" otherwise
vocab2index.get("?", vocab2index["UNK"])

In [ ]:
np.array([vocab2index.get(w, vocab2index["UNK"]) for w in X_train[0].split()])

In [ ]:
def encode_sentence(s, N=40):
    enc = np.zeros(N, dtype=np.int32)
    enc1 = np.array([vocab2index.get(w, vocab2index["UNK"]) for w in s.split()])
    l = min(N, len(enc1))
    enc[:l] = enc1[:l]
    return enc, l

In [ ]:
encode_sentence(X_train[0])

In [ ]:
x_train_len = np.minimum(x_train_len, 40)
x_val_len = np.minimum(x_val_len, 40)

In [ ]:
x_train = np.vstack([encode_sentence(x) for x in X_train])
x_train.shape

In [ ]:
x_val = np.vstack([encode_sentence(x) for x in X_val])
x_val.shape

## Embedding layer
Most deep learning models use a dense vectors of real numbers as representation of words (word embeddings), as opposed to a one-hot encoding representations. The module torch.nn.Embedding is used to represent word embeddings. It takes two arguments: the vocabulary size, and the dimensionality of the embeddings. The embeddings are initialized with random vectors.

In [ ]:
# an Embedding module containing 10 words with embedding size 4
# embedding will be initialized at random
embed = nn.Embedding(10, 4, padding_idx=0)
embed.weight

Note that the `padding_idx` has embedding vector 0.

In [ ]:
# given a list of ids we can "look up" the embedding corresponing to each id
# can you see that some vectors are the same?
a = torch.LongTensor([[1,4,1,5,1,0]])
embed(a)

This would be the representation of a sentence with words with indices [1,4,1,5,1] and a padding at the end. Bellow we have an example in which we have two sentences. the first sentence has length 3 and the last sentence has length 2. In order to use a tensor we use padding at the end of the second sentence.

In [ ]:
a = torch.LongTensor([[1,4,1], [1,3,0]])

Our model takes an average of the word embedding of each word. Here is how we do it.

In [ ]:
s = torch.FloatTensor([3, 2]) # here is the size of the vector

In [ ]:
embed(a)

In [ ]:
embed(a).sum(dim=1)

In [ ]:
sum_embs = embed(a).sum(dim=1)
sum_embs/ s.view(s.shape[0], 1)

## Continuous Bag of Words Model

In [ ]:
class CBOW(nn.Module):
    def __init__(self, vocab_size, emb_size=100):
        super(CBOW, self).__init__()
        self.word_emb = nn.Embedding(vocab_size, emb_size, padding_idx=0)
        self.linear = nn.Linear(emb_size, 1)

    def forward(self, x, s):
        x = self.word_emb(x)
        x = x.sum(dim=1)/ s
        x = self.linear(x)
        return x

# Training the CBOW model

In [ ]:
V = len(words)
model = CBOW(vocab_size=V, emb_size=50)
print(V)

In [ ]:
class SubjectivityDataset(Dataset):
    def __init__(self, X, y):
        self.x = X
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        x = self.x[idx]
        x, s = encode_sentence(x)
        return x, self.y[idx], s

train_ds = SubjectivityDataset(X_train, y_train)
valid_ds = SubjectivityDataset(X_val, y_val)

In [ ]:
train_dl = DataLoader(train_ds, batch_size=5, shuffle=True)
x, y, s = next(iter(train_dl))

In [ ]:
x, y, s

In [ ]:
model = CBOW(vocab_size=V, emb_size=50)

In [ ]:
train_dl = DataLoader(train_ds, batch_size=500, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=500)

In [ ]:
def valid_metrics(model):
    model.eval()
    total = 0
    sum_loss = 0
    correct = 0
    for x, y, s in valid_dl:
        x = x.long()  #.cuda()
        y = y.float().unsqueeze(1)
        s = s.float().view(s.shape[0], 1)
        batch = y.shape[0]
        out = model(x, s)
        loss = F.binary_cross_entropy_with_logits(out, y)
        sum_loss += batch*(loss.item())
        total += batch
        pred = (out > 0).float()
        correct += (pred == y).float().sum().item()
    val_loss = sum_loss/total
    val_acc = correct/total
    return val_loss, val_acc

In [ ]:
def train_epocs(model, optimizer, epochs=10):
    for i in range(epochs):
        model.train()
        total_loss = 0
        total = 0
        for x, y, s in train_dl:
            x = x.long()
            y = y.float().unsqueeze(1)
            s = s.float().view(s.shape[0], 1)
            out = model(x, s)
            loss = F.binary_cross_entropy_with_logits(out, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += x.size(0)*loss.item()
            total += x.size(0)
        train_loss = total_loss/total
        val_loss, val_accuracy = valid_metrics(model)

        print("train_loss %.3f val_loss %.3f val_accuracy %.3f" % (
            train_loss, val_loss, val_accuracy))

In [ ]:
model = CBOW(vocab_size=V, emb_size=50)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
train_epocs(model, optimizer, epochs=10)

## Using pre-trained embeddings

To get glove pre-trained embeddings:
    `wget http://nlp.stanford.edu/data/glove.6B.zip` or use this function:

In [ ]:
def unpack_glove():
    ! wget http://nlp.stanford.edu/data/glove.6B.zip
    ! mkdir data
    ! unzip glove.6B.zip -C data

# unpack_glove()

In [ ]:
! head -2 data/glove.6B.50d.txt

We would like to initialize the embeddings from our model with the pre-trained Glove embeddings. After initializing we should "freeze" the embeddings at least initially. The rationale is that we first want the network to learn weights for the other parameters that were randomly initialize. After that phase we could finetune the embeddings to our task.

`embed.weight.requires_grad = False` freezes the embedding parameters.

The following code initializes the embedding. Here `V` is the vocabulary size and `emb_size` is the embedding size. `pretrained_weight` is a numpy matrix of shape `(V, emb_size)`.

In [ ]:
def loadGloveModel(gloveFile=PATH/"glove.6B.50d.txt"):
    """ Loads word vectors into a dictionary."""
    f = open(gloveFile,'r')
    word_vecs = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        word_vecs[word] = np.array([float(val) for val in splitLine[1:]])
    return word_vecs

In [ ]:
word_vecs = loadGloveModel()

In [ ]:
# let compute the vocab again
word_count = get_vocab(X_train)

In [ ]:
print(len(word_vecs.keys()), len(word_count.keys()))

In [ ]:
def delete_rare_words(word_vecs, word_count, min_df=4):
    """ Deletes rare words from word_count

    Deletes words from word_count if they are not in word_vecs
    and don't have at least min_df occurrencies in word_count.
    """
    words_delete = []
    for word in word_count:
        if word_count[word] < min_df and word not in word_vecs:
            words_delete.append(word)
    for word in words_delete: word_count.pop(word)
    return word_count

In [ ]:
word_count = delete_rare_words(word_vecs, word_count)
print(len(word_count.keys()))

In [ ]:
def create_embedding_matrix(word_vecs, word_count, min_df=4, emb_size=50):
    """Creates embedding matrix from word vectors. """
    word_count = delete_rare_words(word_vecs, word_count, min_df)
    V = len(word_count.keys()) + 2
    vocab2index = {}
    W = np.zeros((V, emb_size), dtype="float32")
    vocab = ["", "UNK"]
    # adding a vector for padding
    W[0] = np.zeros(emb_size, dtype='float32')
    # adding a vector for rare words
    W[1] = np.random.uniform(-0.25, 0.25, emb_size)
    vocab2index["UNK"] = 1
    i = 2
    for word in word_count:
        if word in word_vecs:
            W[i] = word_vecs[word]
            vocab2index[word] = i
            vocab.append(word)
            i += 1
        else:
            W[i] = np.random.uniform(-0.25,0.25, emb_size)
            vocab2index[word] = i
            vocab.append(word)
            i += 1
    return W, np.array(vocab), vocab2index

In [ ]:
pretrained_weight, vocab, vocab2index = create_embedding_matrix(word_vecs, word_count)

In [ ]:
len(pretrained_weight)

In [ ]:
# creating an embedding matrix with Glove embeddings
emb_size = 50
V = len(pretrained_weight)
emb = nn.Embedding(V, emb_size)
emb.weight.data.copy_(torch.from_numpy(pretrained_weight))

## Model with pre-trained embeddings

In [ ]:
class CBOW(nn.Module):
    def __init__(self, vocab_size, emb_size=50, glove_weights=None):
        super(CBOW, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size, padding_idx=0)
        if glove_weights is not None:
            self.embedding.weight.data.copy_(torch.from_numpy(glove_weights))
            self.embedding.weight.requires_grad = False ## freeze embeddings

        self.linear = nn.Linear(emb_size, 1)

    def forward(self, x, s):
        x = self.embedding(x)
        x = x.sum(dim=1)/ s
        x = self.linear(x)
        return x

In [ ]:
def set_learning_rate(optimizer, lr):
    """Changing learning rates without creating a new optimizer"""
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [ ]:
model = CBOW(V, emb_size=50, glove_weights=pretrained_weight)
parameters = filter(lambda p: p.requires_grad, model.parameters())
optimizer = torch.optim.Adam(parameters, lr=0.01)

In [ ]:
train_epocs(model, optimizer, epochs=5)

In [ ]:
# unfreezing the embeddings
model.embedding.weight.requires_grad = True

In [ ]:
parameters = filter(lambda p: p.requires_grad, model.parameters())
optimizer = torch.optim.Adam(parameters, lr=0.01)

In [ ]:
train_epocs(model, optimizer, epochs=5)

In [ ]:
set_learning_rate(optimizer, 0.001)
train_epocs(model, optimizer, epochs=10)